##  Imports

## setup openai

In [1]:
import sys
sys.path.append("llmapslab")
import os
import json
from  openai import OpenAI
from langchain_openai import ChatOpenAI

### Open AI api access setup 
- Api access account is not same as the chatgpt account. 
- setup your project and api key from here https://platform.openai.com/playground/chat?models=gpt-4o-mini-2024-07-18
- create a new api key and copy and save it before closing the pop up window. Once the window is closed 
 the key is no more accessable.
- before making call to openai ensure you have  balance and keep track of your cost
- come back to the playground and try in the gui a simple chat to ensure chat is working
https://platform.openai.com/organization/usage

In [2]:
with open('secrets.json', 'r') as jsonfile:
    configs = json.load(jsonfile)

client = OpenAI(api_key=configs.get('openai_api_key'),
                # organization='Personal',
                # project='proj_DqdKUU38qOGVj9Qxdq6UlXcD',
               )

In [3]:
def call_openai_api(client, messages, model="gpt-3.5-turbo", max_retries=5, wait_time=5):
    retries = 0
    while retries < max_retries:
        try:
            # Make a request to the OpenAI ChatCompletion API
           response = client.chat.completions.create(
              model="gpt-4o-mini",
              messages=messages,
           )
           return response
        except client.error.RateLimitError:
            print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            retries += 1
    raise Exception("Max retries exceeded. Please check your plan and billing details.")

# Define the conversation messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]

# Call the API
response = call_openai_api(client, messages)

In [4]:
print(response)

ChatCompletion(id='chatcmpl-9ofhR9VHlYfDwyVLy8qzh2aGkvhCq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None))], created=1721863721, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_661538dc1f', usage=CompletionUsage(completion_tokens=7, prompt_tokens=24, total_tokens=31))


In [5]:
response.choices[0].message

ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None)

In [6]:
response.choices[0].message.content

'The capital of France is Paris.'

## langchain ChatOpenAI call 

In [7]:
openai_api_key = configs.get('openai_api_key')
messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]

llm = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o"
    )
response = llm.invoke(messages)
print(response)

content="J'aime la programmation." response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 28, 'total_tokens': 33}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None} id='run-7a48c0f6-c0d1-4b96-9228-ad1f90f64173-0' usage_metadata={'input_tokens': 28, 'output_tokens': 5, 'total_tokens': 33}


In [8]:
response.content

"J'aime la programmation."

In [9]:
messages = [
    (
        "system",
        "You are an assistant",
    ),
    ("human", "generate a code to read excel"),
]

llm = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o"
    )
response = llm.invoke(messages)
print(response)

content="To read Excel files in Python, you can use the `pandas` library, which provides a convenient interface for working with data in various formats, including Excel. Below is a simple example of how to read an Excel file using `pandas`. Make sure you have the library installed; if you haven't installed it yet, you can do so using pip:\n\n```bash\npip install pandas openpyxl\n```\n\n### Example Code to Read an Excel File\n\nHere's a basic example to read an Excel file named `example.xlsx`:\n\n```python\nimport pandas as pd\n\n# Specify the path to your Excel file\nfile_path = 'example.xlsx'\n\n# Read the Excel file\n# You can specify the sheet name or index (0 for the first sheet)\ndf = pd.read_excel(file_path, sheet_name=0)  # or 'Sheet1' for a specific sheet\n\n# Display the contents of the DataFrame\nprint(df)\n```\n\n### Explanation:\n- Import the `pandas` library.\n- Specify the path to your Excel file.\n- Use `pd.read_excel()` to read the Excel file into a DataFrame. You can 

In [10]:
print(response.content)

To read Excel files in Python, you can use the `pandas` library, which provides a convenient interface for working with data in various formats, including Excel. Below is a simple example of how to read an Excel file using `pandas`. Make sure you have the library installed; if you haven't installed it yet, you can do so using pip:

```bash
pip install pandas openpyxl
```

### Example Code to Read an Excel File

Here's a basic example to read an Excel file named `example.xlsx`:

```python
import pandas as pd

# Specify the path to your Excel file
file_path = 'example.xlsx'

# Read the Excel file
# You can specify the sheet name or index (0 for the first sheet)
df = pd.read_excel(file_path, sheet_name=0)  # or 'Sheet1' for a specific sheet

# Display the contents of the DataFrame
print(df)
```

### Explanation:
- Import the `pandas` library.
- Specify the path to your Excel file.
- Use `pd.read_excel()` to read the Excel file into a DataFrame. You can specify the `sheet_name` parameter t

## Handcrafted Agent 

#### what is agent doing
- calling model with a prompt
- maintianing a history of conversation

In [15]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": sysytem})

    def __call__(self, messages):
        self.messages.append({"role": "user", "content": sysytem})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completion.create(
            model="gpt-4o-mini-2024-07-18",
            temparature=0,
            messages=self.messages)
        return completion.choices[0].messages.content

#### A ReAct type prompt for agent 
- Thought:
  - user asks a question to model
  - model think about a plan of action from the available actions
   - Available Actions:
     - name of the python function: parameters of the function
     - identifying the parameters correctly is critical
- Action:
  - from the free flow text identify the parameters for  the action
  - return the model output as ACtion: function name: paramer
  - This output will be parsed by the user or another program later to call the function
  - also output 'PAUSE'. Why is this required ? may be external program can use this somehow.
- Observation:
  - assume that the function was called and result was obrained by user or by anothe program
  - the model is now again called with the result along with the history
  - the model now get to see:
     - The original quesiton from the user
     - The actions that it had suggested in terms of function name and its parameters
     - the  result from the action
- Answer: provide the final answer from the observation and the history of messages as context
- Demonstration: give a task demonstration

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()